In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I built this script to check on one of the terms in the script me and steffen have been trying to write. 

$ \langle ij|ab\rangle\langle jk|bc\rangle\langle ki|ca\rangle $

$= [ia|jb][jb|kc][kc|ia]$

In [2]:
def enumerator(eri,i,j,k,a,b,c):
    return eri[i,a,j,b]*eri[j,b,k,c]*eri[k,c,i,a]

$( \epsilon_{jb} - \epsilon_{i a})(\epsilon_{k c} - \epsilon_{i a})$

In [3]:
def denominator_1(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]-eigval[i]+eigval[a])*(eigval[k]-eigval[c]-eigval[i]+eigval[a]))

$( \epsilon_{j b} - \epsilon_{i a})(\epsilon_{k c} + \epsilon_{i a})$

In [4]:
def denominator_2(eigval,i,j,k,a,b,c):
    res = ((eigval[j]-eigval[b]-eigval[i]+eigval[a])*(eigval[k]-eigval[c]+eigval[i]+eigval[a]))
    if res == 0:
        # the issue comes from all p orbitals being degenrate, a feature of the spherical symmetry
        print("Error, result 0")
        print(f"pair {j}{b} - {i}{a} is:")
        print(eigval[j]-eigval[b]-eigval[i]+eigval[a])
        print(f" {j}-{b} and {i}-{a} is:")
        print(eigval[j]-eigval[b],eigval[i]-eigval[a])
        print(f" {j},{b} and {i},{a} is:")
        print(eigval[j],eigval[b],eigval[i],eigval[a])
    return res

$( \epsilon_{j b} + \epsilon_{i a})(\epsilon_{k c} - \epsilon_{i a})$

In [5]:
def denominator_3(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]+eigval[i]+eigval[a])*(eigval[k]-eigval[c]-eigval[i]+eigval[a]))

$( \epsilon_{j b} + \epsilon_{i a})(\epsilon_{k c} + \epsilon_{i a})$

In [6]:
def denominator_4(eigval,i,j,k,a,b,c):
    return ((eigval[j]-eigval[b]+eigval[i]-eigval[a])*(eigval[k]-eigval[c]+eigval[i]-eigval[a]))

Term 1:
$$\frac{ \sum_{ia,jb,kc} \langle ij|ab\rangle\langle jk|bc\rangle\langle ki|ca\rangle }{( \epsilon_{j b} - \epsilon_{i a})(\epsilon_{k c} - \epsilon_{i a})} $$

where $i,j,k$ are occupied and $a,b,c$ are unoccupied orbitals and $ia \neq  jb \neq kc$

In [7]:
def Term_1(eri,eigval,M,N):
    T0_array, T1_array, T2_array, T3_array = [],[],[],[]
    T = 0 
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                for a in range(N,M):
                    for b in range(N,M):
                        for c in range(N,M):
                            if [i, a] != [j, b] and [j, b] != [k,c] :
                                #print(  f"{denominator_2(eigval,i,j,k,a,b,c):2.2f}",\
                                #  f"{denominator_3(eigval,i,j,k,a,b,c):2.2f}" )
                                T0_array.append(denominator_1(eigval,i,j,k,a,b,c))
                                T1_array.append(denominator_2(eigval,i,j,k,a,b,c))
                                T2_array.append(denominator_3(eigval,i,j,k,a,b,c))
                                T3_array.append(denominator_4(eigval,i,j,k,a,b,c))


                                

                                # print(i,a,j,b,k,c ,\
                                #   f"{denominator_1(eigval,i,j,k,a,b,c):2.2f}", f"{denominator_2(eigval,i,j,k,a,b,c):2.2f}",\
                                #   f"{denominator_3(eigval,i,j,k,a,b,c):2.2f}", f"{denominator_4(eigval,i,j,k,a,b,c):2.2f}")
    # i was hoping some of these terms would maybe add up to 0 or add up to same numbers of opposite signs
    print(sum(T0_array), sum(T1_array), sum(T2_array), sum(T3_array))


    return T

In [8]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
mol.basis = "cc-pvdz"
#mol.basis = "6-31g" 
mol.build()

In [9]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
h = mf.get_hcore()
C = mf.mo_coeff
h1 = C.T@h@C
epsilon = mf.mo_energy

converged SCF energy = -2.85516047724274


In [10]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
eri = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [11]:
print(eri.shape)

(5, 5, 5, 5)


In [12]:
N = mol.nelec[0]
M = mol.nao
print(M,N)
print(epsilon)
print(Term_1(eri,epsilon,M,N))

5 1
[-0.91414793  1.3974417   2.52437203  2.52437203  2.52437203]
Error, result 0
pair 03 - 02 is:
0.0
 0-3 and 0-2 is:
-3.438519951821347 -3.438519951821347
 0,3 and 0,2 is:
-0.9141479259458158 2.524372025875531 -0.9141479259458158 2.524372025875531
Error, result 0
pair 03 - 02 is:
0.0
 0-3 and 0-2 is:
-3.438519951821347 -3.438519951821347
 0,3 and 0,2 is:
-0.9141479259458158 2.524372025875531 -0.9141479259458158 2.524372025875531
Error, result 0
pair 03 - 02 is:
0.0
 0-3 and 0-2 is:
-3.438519951821347 -3.438519951821347
 0,3 and 0,2 is:
-0.9141479259458158 2.524372025875531 -0.9141479259458158 2.524372025875531
Error, result 0
pair 04 - 02 is:
0.0
 0-4 and 0-2 is:
-3.438519951821347 -3.438519951821347
 0,4 and 0,2 is:
-0.9141479259458158 2.524372025875531 -0.9141479259458158 2.524372025875531
Error, result 0
pair 04 - 02 is:
0.0
 0-4 and 0-2 is:
-3.438519951821347 -3.438519951821347
 0,4 and 0,2 is:
-0.9141479259458158 2.524372025875531 -0.9141479259458158 2.524372025875531
Error, re